# Choosing the best chemical dosage (IC50) per drug per patient
# Uses CNN and simple feedforward neural net
# Use this instead of regression tree IF we want SMILES strings to be a feature to consider. Mixing molecular and patient information together

# LIMITATION NOTES TO EXPRESS TO AUDIENCE:

<b>PROBLEM: 
- We have only 1 IC50 value per cell line x drug. This means there is no variation in IC50 values per cell line x drug. This means demographic information has little affect on our results.¶

<b>FIX:
- CELL LINE IS A PROXY FOR THE PATIENT.
- <mark>IF WE ASSUME THE CELL LINE REPRESENTS THE PATIENT, AND WANT PATIENT DEMOGRAPHICS TO INFLUENCE DRUG RECOMMENDATION or IC50 OUTPUTS, THEN PATIENT DEMOGRAPHICS SHOULD BE MOST IMPORTANT TO IMPUTE CELL LINE. 
- CONSIDER OTHER FEATURES TO IMPROVE ACCURACY: BMI, MENOPAUSE STATUS

<B>USE CASE: Given patient demogprahic information, we give them 35 optional treatments. We want to recommend them at least 5 drugs based on the IC50 values. That means IC50 should be predictor. The doctor should recommend effective drugs. Effective = lower IC50 values.  

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time as time
valid_IC50s = pd.read_csv("valid_IC50s_within_range.csv")
merged_df = pd.read_csv("final_merged.csv")
lucas_df = pd.read_csv("final_mapping.csv")
cell_lines_df = pd.read_csv("data/HarvardCellLines.csv")

/var/folders/s8/ghqk1l4n7n9_w17t7hx21g2m0000gn/T/ipykernel_1855/3521542763.py:7: DtypeWarning: Columns (0,3,4,5,6,7,9,12,15,16,17,18,22,24,26,28,29,30,40,65,66,67,68,69,70,71,72,73,74,75,76,82,83,89,90,91,92,93,96,97,98,99,100,101,102,103,104,105,106,107,108,109,115) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv("final_merged.csv")


In [4]:
valid_IC50s.drop(columns = ['Unnamed: 0', 'N Points'], inplace = True)

In [6]:
cell_lines_df.columns

Index(['HMS LINCS Batch ID', 'HMS LINCS ID', 'Name', 'Alternative Names',
       'LINCS ID', 'Alternative ID', 'Reference Source', 'Organism', 'Organ',
       'Tissue', 'Cell Type', 'Details of Cell Type', 'Donor Sex', 'Donor Age',
       'Donor Ethnicity', 'Donor Health Status', 'Disease', 'Unnamed: 17',
       'Details of Disease', 'Production Details', 'Genetic Modification(s)',
       'Known Mutations', 'Citation Information for Mutations',
       'Verification Reference Profile', 'Growth Properties',
       'Recommended Culture Conditions', 'Relevant Citations', 'Usage Note',
       'Comments', 'Provider', 'Provider Catalog ID', 'Provider Batch ID',
       'Source Information', 'Date Received', 'HMS QC Outcome',
       'Transient Modification(s)', 'Date Publicly Available',
       'Most Recent Update', 'T Stage'],
      dtype='object')

In [8]:
columns = ["HMS LINCS ID", "Name"]
cell_lines_df = cell_lines_df[columns]

In [10]:
cell_lines_df_dict = cell_lines_df.set_index("HMS LINCS ID").to_dict()["Name"]
cell_lines_df_dict

{50008: 'CAL-51',
 50029: 'MCF7',
 50056: 'HME1',
 50057: 'SK-BR-3',
 50058: 'MDA-MB-231',
 50105: 'BT-20',
 50108: 'BT-549',
 50131: 'CAMA-1',
 50205: 'HCC1143',
 50206: 'HCC1395',
 50207: 'HCC1419',
 50208: 'HCC1428',
 50211: 'HCC1806',
 50212: 'HCC1937',
 50213: 'HCC1954',
 50216: 'HCC38',
 50219: 'HCC70',
 50238: 'Hs 578T',
 50327: 'MDA-MB-134-VI',
 50328: 'MDA-MB-157',
 50331: 'MDA-MB-361',
 50333: 'MDA-MB-436',
 50334: 'MDA-MB-453',
 50335: 'MDA-MB-468',
 50541: 'T47D',
 50579: 'HCC1500',
 50583: 'MCF 10A',
 51081: 'SUM1315MO2',
 51082: 'SUM149PT',
 51083: 'SUM159PT',
 51110: 'CAL-120',
 51112: 'CAL-85-1',
 51134: 'PDX1258',
 51135: 'PDX1328',
 51136: 'PDXHCI002'}

In [12]:
def cell_name_map(row, dictionary):
    return dictionary[row["Cell_Line"]]

In [14]:
merged_df["Race"].value_counts()

Race
1.0    5857
2.0     470
4.0     321
0.0      19
5.0      18
3.0      14
6.0       9
8.0       4
7.0       1
Name: count, dtype: int64

In [16]:
# # TEMPORARY IMPUTATION OF RACE BASED OFF PROPORTIONS OF EXISTING RACES.
# # WILL USE AUSTIN'S IMPUTED RACE ANALYSIS LATER.

# # Get value counts as probabilities
# race_dist = merged_df['Race'].value_counts(normalize=True)

# # Get the indices where race is missing
# missing_indices = lucas_df['Cell_Line_Race'].isna()

# # Sample values based on observed distribution
# imputed_values = np.random.choice(race_dist.index, size=missing_indices.sum(), p=race_dist.values)

# # Assign the sampled values to the missing positions
# lucas_df.loc[missing_indices, 'Cell_Line_Race'] = imputed_values


In [18]:
# TEMPORARY IMPUTATION OF RACE BASED OFF PROPORTIONS OF EXISTING RACES.
# WILL USE AUSTIN'S IMPUTED RACE ANALYSIS LATER.

# Get value counts as probabilities
race_dist = merged_df['Race'].value_counts(normalize=True)

# Get the indices where race is missing
missing_indices = merged_df['Race'].isna()

# Sample values based on observed distribution
imputed_values = np.random.choice(race_dist.index, size=missing_indices.sum(), p=race_dist.values)

# Assign the sampled values to the missing positions
merged_df.loc[missing_indices, 'Race'] = imputed_values

In [20]:
def T_stage_by_size(size):
    if size == 0:
        return 0
    if size > 0 and size <= 20:
        return 1
    if size > 20 and size <= 50:
        return 2
    if size > 50:
        return 3

In [22]:
merged_df['T_stage_by_size'] = merged_df.apply(lambda row: row['T Stage'] if pd.notnull(row['T Stage']) else T_stage_by_size(row['Tumor Size']), axis=1)

In [24]:
merged_df['T_stage_by_size'].isna().sum()

134

In [26]:
columns = ['Patient ID', 'Age', 'Race', 'T_stage_by_size']
patients_df = merged_df[columns]

In [28]:
patients_df

,Patient ID,Age,Race,T_stage_by_size
0,Breast_MRI_001,41,2.0,2.0
1,Breast_MRI_001,41,2.0,2.0
2,Breast_MRI_002,38,2.0,2.0
3,Breast_MRI_003,62,1.0,2.0
4,Breast_MRI_003,62,1.0,2.0
...,...,...,...,...
9217,NaN,69,1.0,4.0
9218,NaN,69,1.0,4.0
9219,NaN,69,1.0,4.0
9220,NaN,69,1.0,4.0


In [30]:
patients_df.isna().sum()

Patient ID         1750
Age                   0
Race                  0
T_stage_by_size     134
dtype: int64

In [32]:
valid_IC50s

,Cell Name,Small Molecule Name,EC50 (uM)
0,BT-20,A-1210477,0.005488
1,BT-20,AZD7762,1.650602
2,BT-20,Bleomycin,0.754695
3,BT-20,Buparlisib,1.336570
4,BT-20,Cabozantinib,3.789538
...,...,...,...
821,T47D,Topotecan,0.006967
822,T47D,Torin2,0.004775
823,T47D,Trametinib,0.005605
824,T47D,Volasertib,0.033216


In [34]:
# ASK LUCAS FOR THIS DATA
patients_df = patients_df.copy()

# Assign a random integer between 1 and 34 for each row
patients_df['cell_line'] = np.random.randint(1, 36, size=len(patients_df))

In [36]:
patients_df

,Patient ID,Age,Race,T_stage_by_size,cell_line
0,Breast_MRI_001,41,2.0,2.0,27
1,Breast_MRI_001,41,2.0,2.0,16
2,Breast_MRI_002,38,2.0,2.0,26
3,Breast_MRI_003,62,1.0,2.0,27
4,Breast_MRI_003,62,1.0,2.0,18
...,...,...,...,...,...
9217,NaN,69,1.0,4.0,4
9218,NaN,69,1.0,4.0,3
9219,NaN,69,1.0,4.0,10
9220,NaN,69,1.0,4.0,13


In [39]:
# ASK TEAM TO HELP IMPUTE MISSING T-STAGE OR DROP THEIR ROWS 
# MAYBE ASK AUSTIN FOR SIMILAR WORKFLOW USED FOR RACE BUT FOR T-STAGE 
patients_df.isna().sum()

Patient ID         1750
Age                   0
Race                  0
T_stage_by_size     134
cell_line             0
dtype: int64

### Each patient needs a one hot encoded version of the chemical for the cell line they represent...

In [42]:
valid_IC50s

,Cell Name,Small Molecule Name,EC50 (uM)
0,BT-20,A-1210477,0.005488
1,BT-20,AZD7762,1.650602
2,BT-20,Bleomycin,0.754695
3,BT-20,Buparlisib,1.336570
4,BT-20,Cabozantinib,3.789538
...,...,...,...
821,T47D,Topotecan,0.006967
822,T47D,Torin2,0.004775
823,T47D,Trametinib,0.005605
824,T47D,Volasertib,0.033216


In [44]:
# Get unique cell line names
unique_cell_lines = valid_IC50s['Cell Name'].unique()

# Create a mapping from cell line name to number (1 to 34)
cell_line_map = {name: i+1 for i, name in enumerate(unique_cell_lines)}

# Preview the result
print(cell_line_map)

valid_IC50s['Cell_Name_Mapped'] = valid_IC50s['Cell Name'].map(cell_line_map)

{'BT-20': 1, 'BT-549': 2, 'CAL-120': 3, 'CAL-51': 4, 'CAL-85-1': 5, 'CAMA-1': 6, 'HCC1143': 7, 'HCC1395': 8, 'HCC1419': 9, 'HCC1428': 10, 'HCC1500': 11, 'HCC1806': 12, 'HCC1937': 13, 'HCC1954': 14, 'HCC38': 15, 'HCC70': 16, 'HME1': 17, 'Hs 578T': 18, 'MCF 10A': 19, 'MCF7': 20, 'MDA-MB-134-VI': 21, 'MDA-MB-157': 22, 'MDA-MB-231': 23, 'MDA-MB-361': 24, 'MDA-MB-436': 25, 'MDA-MB-453': 26, 'MDA-MB-468': 27, 'PDX1258': 28, 'PDX1328': 29, 'PDXHCI002': 30, 'SK-BR-3': 31, 'SUM1315MO2': 32, 'SUM149PT': 33, 'SUM159PT': 34, 'T47D': 35}


In [46]:
valid_IC50s

,Cell Name,Small Molecule Name,EC50 (uM),Cell_Name_Mapped
0,BT-20,A-1210477,0.005488,1
1,BT-20,AZD7762,1.650602,1
2,BT-20,Bleomycin,0.754695,1
3,BT-20,Buparlisib,1.336570,1
4,BT-20,Cabozantinib,3.789538,1
...,...,...,...,...
821,T47D,Topotecan,0.006967,35
822,T47D,Torin2,0.004775,35
823,T47D,Trametinib,0.005605,35
824,T47D,Volasertib,0.033216,35


In [48]:
# One-hot encode 'Small Molecule Name'
one_hot_encoded = pd.get_dummies(valid_IC50s['Small Molecule Name'], dtype='int')

# Concatenate the one-hot encoded columns back to the original DataFrame
valid_IC50s_encoded = pd.concat([valid_IC50s, one_hot_encoded], axis=1)

In [50]:
valid_IC50s_encoded

,Cell Name,Small Molecule Name,EC50 (uM),Cell_Name_Mapped,A-1210477,ABT-737,AZD7762,Abemaciclib,Alpelisib,Bleomycin,...,Saracatinib,TGX221,Taselisib,Taxol,Tivantinib,Topotecan,Torin2,Trametinib,Volasertib,Vorinostat
0,BT-20,A-1210477,0.005488,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BT-20,AZD7762,1.650602,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BT-20,Bleomycin,0.754695,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,BT-20,Buparlisib,1.336570,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BT-20,Cabozantinib,3.789538,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,T47D,Topotecan,0.006967,35,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
822,T47D,Torin2,0.004775,35,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
823,T47D,Trametinib,0.005605,35,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
824,T47D,Volasertib,0.033216,35,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [52]:
patients_df

,Patient ID,Age,Race,T_stage_by_size,cell_line
0,Breast_MRI_001,41,2.0,2.0,27
1,Breast_MRI_001,41,2.0,2.0,16
2,Breast_MRI_002,38,2.0,2.0,26
3,Breast_MRI_003,62,1.0,2.0,27
4,Breast_MRI_003,62,1.0,2.0,18
...,...,...,...,...,...
9217,NaN,69,1.0,4.0,4
9218,NaN,69,1.0,4.0,3
9219,NaN,69,1.0,4.0,10
9220,NaN,69,1.0,4.0,13


In [54]:
valid_IC50s

,Cell Name,Small Molecule Name,EC50 (uM),Cell_Name_Mapped
0,BT-20,A-1210477,0.005488,1
1,BT-20,AZD7762,1.650602,1
2,BT-20,Bleomycin,0.754695,1
3,BT-20,Buparlisib,1.336570,1
4,BT-20,Cabozantinib,3.789538,1
...,...,...,...,...
821,T47D,Topotecan,0.006967,35
822,T47D,Torin2,0.004775,35
823,T47D,Trametinib,0.005605,35
824,T47D,Volasertib,0.033216,35


In [56]:
# Step 1: Make sure both dataframes have a matching cell line column
# Map Cell Name (string) to cell_line_id (integer) in the drug data
# valid_IC50s_encoded['cell_line_id'] = valid_IC50s_encoded['Cell Name'].map(cell_line_map)

# Step 2: Merge patients with drug data on cell_line == cell_line_id
# This will replicate each patient row once per matching drug for their cell line
patient_drug_df = patients_df.merge(
    valid_IC50s,
    left_on='cell_line',
    right_on='Cell_Name_Mapped',
    how='inner'  # use 'inner' to only keep rows with valid matches
)

In [58]:
patient_drug_df.drop(columns="cell_line", inplace=True)

In [60]:
race_dict = {0:'N/A', 1:"white", 2:"black", 3:"asian", 4:"native", 5:"hispanic", 6:"multi", 7:"hawa", 8:"amer indian"}

In [62]:
patient_drug_df

,Patient ID,Age,Race,T_stage_by_size,Cell Name,Small Molecule Name,EC50 (uM),Cell_Name_Mapped
0,Breast_MRI_001,41,2.0,2.0,MDA-MB-468,AZD7762,0.608155,27
1,Breast_MRI_001,41,2.0,2.0,MDA-MB-468,Abemaciclib,3.897254,27
2,Breast_MRI_001,41,2.0,2.0,MDA-MB-468,Bleomycin,1.683910,27
3,Breast_MRI_001,41,2.0,2.0,MDA-MB-468,Buparlisib,1.169445,27
4,Breast_MRI_001,41,2.0,2.0,MDA-MB-468,Ceritinib,1.386082,27
...,...,...,...,...,...,...,...,...
217525,NaN,69,1.0,4.0,HCC1143,Topotecan,0.004448,7
217526,NaN,69,1.0,4.0,HCC1143,Torin2,0.061084,7
217527,NaN,69,1.0,4.0,HCC1143,Trametinib,0.013374,7
217528,NaN,69,1.0,4.0,HCC1143,Volasertib,0.006007,7


In [64]:
SmallMolecules = pd.read_csv("data/HarvardSmallMolecules.csv")
columns = ['Name', 'Molecular Mass', 'SMILES']
SmallMolecules = SmallMolecules[columns]
SmallMolecules

,Name,Molecular Mass,SMILES
0,AZD7762,362.12,C1C[C@H](CNC1)NC(=O)C2=C(C=C(S2)C3=CC(=CC=C3)F...
1,Neratinib,556.20,CCOC1=C(C=C2C(=C1)N=CC(=C2NC3=CC(=C(C=C3)OCC4=...
2,Dasatinib,487.16,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=NC(=NC(...
3,Saracatinib,541.21,CN1CCN(CC1)CCOC2=CC(=C3C(=C2)N=CN=C3NC4=C(C=CC...
4,Pictilisib,513.16,CS(=O)(=O)N1CCN(CC1)CC2=CC3=C(S2)C(=NC(=N3)C4=...
5,Palbociclib,447.24,CC1=C(C(=O)N(C2=NC(=NC=C12)NC3=NC=C(C=C3)N4CCN...
6,Torin2,432.12,C1=CC(=CC(=C1)N2C(=O)C=CC3=CN=C4C=CC(=CC4=C32)...
7,Taxol,853.33,CC1=C2[C@H](C(=O)[C@@]3([C@H](C[C@@H]4[C@](C3[...
8,Tivantinib,369.15,C1CC2=CC=CC3=C2N(C1)C=C3[C@H]4[C@@H](C(=O)NC4=...
9,Trametinib,615.08,CC1=C2C(=C(N(C1=O)C)NC3=C(C=C(C=C3)I)F)C(=O)N(...


In [66]:
SMILES_Map = SmallMolecules[["Name", "SMILES"]].set_index("Name").to_dict()['SMILES']

In [68]:
SMILES_Map

{'AZD7762': 'C1C[C@H](CNC1)NC(=O)C2=C(C=C(S2)C3=CC(=CC=C3)F)NC(=O)N',
 'Neratinib': 'CCOC1=C(C=C2C(=C1)N=CC(=C2NC3=CC(=C(C=C3)OCC4=CC=CC=N4)Cl)C#N)NC(=O)C=CCN(C)C',
 'Dasatinib': 'CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=NC(=NC(=C3)N4CCN(CC4)CCO)C',
 'Saracatinib': 'CN1CCN(CC1)CCOC2=CC(=C3C(=C2)N=CN=C3NC4=C(C=CC5=C4OCO5)Cl)OC6CCOCC6',
 'Pictilisib': 'CS(=O)(=O)N1CCN(CC1)CC2=CC3=C(S2)C(=NC(=N3)C4=C5C=NNC5=CC=C4)N6CCOCC6',
 'Palbociclib': 'CC1=C(C(=O)N(C2=NC(=NC=C12)NC3=NC=C(C=C3)N4CCNCC4)C5CCCC5)C(=O)C',
 'Torin2': 'C1=CC(=CC(=C1)N2C(=O)C=CC3=CN=C4C=CC(=CC4=C32)C5=CN=C(C=C5)N)C(F)(F)F',
 'Taxol': 'CC1=C2[C@H](C(=O)[C@@]3([C@H](C[C@@H]4[C@](C3[C@@H]([C@@](C2(C)C)(C[C@@H]1OC(=O)[C@@H]([C@H](C5=CC=CC=C5)NC(=O)C6=CC=CC=C6)O)O)OC(=O)C7=CC=CC=C7)(CO4)OC(=O)C)O)C)OC(=O)C',
 'Tivantinib': 'C1CC2=CC=CC3=C2N(C1)C=C3[C@H]4[C@@H](C(=O)NC4=O)C5=CNC6=CC=CC=C65',
 'Trametinib': 'CC1=C2C(=C(N(C1=O)C)NC3=C(C=C(C=C3)I)F)C(=O)N(C(=O)N2C4=CC(=CC=C4)NC(=O)C)C5CC5',
 'Olaparib': 'C1CC1C(=O)N2CCN(CC2)C(=O)C3=C(

In [70]:
def map_smiles(row, dictionary):
    return dictionary[row["Small Molecule Name"]]

In [72]:
patient_drug_df["SMILES"] = patient_drug_df.apply(map_smiles,dictionary = SMILES_Map,axis=1)

In [73]:
patient_drug_df

,Patient ID,Age,Race,T_stage_by_size,Cell Name,Small Molecule Name,EC50 (uM),Cell_Name_Mapped,SMILES
0,Breast_MRI_001,41,2.0,2.0,MDA-MB-468,AZD7762,0.608155,27,C1C[C@H](CNC1)NC(=O)C2=C(C=C(S2)C3=CC(=CC=C3)F...
1,Breast_MRI_001,41,2.0,2.0,MDA-MB-468,Abemaciclib,3.897254,27,CCN1CCN(CC1)CC2=CN=C(C=C2)NC3=NC=C(C(=N3)C4=CC...
2,Breast_MRI_001,41,2.0,2.0,MDA-MB-468,Bleomycin,1.683910,27,CC1=C(N=C(N=C1N)[C@H](CC(=O)N)NC[C@@H](C(=O)N)...
3,Breast_MRI_001,41,2.0,2.0,MDA-MB-468,Buparlisib,1.169445,27,C1COCCN1C2=NC(=NC(=C2)C3=CN=C(C=C3C(F)(F)F)N)N...
4,Breast_MRI_001,41,2.0,2.0,MDA-MB-468,Ceritinib,1.386082,27,CC1=CC(=C(C=C1C2CCNCC2)OC(C)C)NC3=NC=C(C(=N3)N...
...,...,...,...,...,...,...,...,...,...
217525,NaN,69,1.0,4.0,HCC1143,Topotecan,0.004448,7,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=C(C3=C2)N=C5C=...
217526,NaN,69,1.0,4.0,HCC1143,Torin2,0.061084,7,C1=CC(=CC(=C1)N2C(=O)C=CC3=CN=C4C=CC(=CC4=C32)...
217527,NaN,69,1.0,4.0,HCC1143,Trametinib,0.013374,7,CC1=C2C(=C(N(C1=O)C)NC3=C(C=C(C=C3)I)F)C(=O)N(...
217528,NaN,69,1.0,4.0,HCC1143,Volasertib,0.006007,7,O=C(N[C@H]1CC[C@H](N2CCN(CC3CC3)CC2)CC1)C4=CC=...


# <MARK>AUSTIN OR TEAMMATES PLEASE START CHECKING PAST HERE: 

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [79]:
# Separate features (X) and target (y)
X = patient_drug_df.drop(columns=['Patient ID', 'EC50 (uM)', 'Small Molecule Name', 'Cell_Name_Mapped'])  # Drop EC50 and non-features
y = patient_drug_df['EC50 (uM)']  # EC50 is the target

# Standardize numerical features
scaler = StandardScaler()
X[['Age']] = scaler.fit_transform(X[['Age']]) 

# Need to one-hot encode categorical variables: Race, Drug name, Cell line name
X = pd.get_dummies(X, columns=['Cell Name', 'Race', 'T_stage_by_size'],dtype='int', drop_first=False)

In [81]:
# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [83]:
X_train

,Age,SMILES,Cell Name_BT-20,Cell Name_BT-549,Cell Name_CAL-120,Cell Name_CAL-51,Cell Name_CAL-85-1,Cell Name_CAMA-1,Cell Name_HCC1143,Cell Name_HCC1395,...,Race_4.0,Race_5.0,Race_6.0,Race_7.0,Race_8.0,T_stage_by_size_0.0,T_stage_by_size_1.0,T_stage_by_size_2.0,T_stage_by_size_3.0,T_stage_by_size_4.0
35858,-1.433849,C1CC2=CC=CC3=C2N(C1)C=C3[C@H]4[C@@H](C(=O)NC4=...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
90906,0.183106,CCNC(=O)C1=C(C(=C2C=C(C(=CC2=O)O)C(C)C)ON1)C3=...,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
12817,-0.958274,O=C(N[C@H]1CC[C@H](N2CCN(CC3CC3)CC2)CC1)C4=CC=...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
157639,1.704947,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
157296,-1.338734,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=C(C3=C2)N=C5C=...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0.658682,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=C(C3=C2)N=C5C=...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
103694,0.563567,C[C@@H]1C[C@H](C2=C1C(=NC=N2)N3CCN(CC3)C(=O)[C...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
131932,-2.099655,CC1=C2[C@H](C(=O)[C@@]3([C@H](C[C@@H]4[C@](C3[...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
146867,1.324487,CC1=CC(=C(C=C1C2CCNCC2)OC(C)C)NC3=NC=C(C(=N3)N...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [85]:
import re
# MAYBE SWITCH THIS OUT WITH CHEMBERTA/TRANSFORMER FOR TOKENIZATION 
def tokenize_smiles(smiles):
    """
    Tokenizes a SMILES string into a list of atomic or multi-char tokens.
    Handles 'Cl', 'Br', 'nH', etc.
    """
    pattern = r"(\[[^\[\]]{1,6}\]|Br|Cl|Si|Se|@@?|=|#|\(|\)|\.|\/|\\|\+|\-|\d+|[A-Z][a-z]?|[a-z])"
    return re.findall(pattern, smiles)

In [87]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# OTHER ALTERNATIVE FOR TOKENIZATION 

def build_vocab(smiles_list):
    tokens = sorted(set(char for sm in smiles_list for char in tokenize_smiles(sm)))
    token_to_idx = {token: i + 1 for i, token in enumerate(tokens)}  # 0 = padding
    return token_to_idx

# Convert SMILES to indices
def smiles_to_indices(smiles, token_to_idx, max_len=120):
    tokens = tokenize_smiles(smiles)
    token_ids = [token_to_idx.get(tok, 0) for tok in tokens][:max_len]
    padded = token_ids + [0] * (max_len - len(token_ids))
    return padded


In [98]:
# TWO NEURAL NETS: CNN and FEED FORWARD NEURAL NETWORK

# CNN 
class SMILESEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim=32, out_channels=64, kernel_size=3):
        super(SMILESEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        self.conv1 = nn.Conv1d(in_channels=embed_dim, out_channels=out_channels, kernel_size=kernel_size, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.AdaptiveMaxPool1d(1)  # Get fixed-size output
     
    def forward(self, x):
        x = self.embedding(x)         # (batch, seq_len, embed_dim)
        x = x.permute(0, 2, 1)        # → (batch, embed_dim, seq_len)
        x = self.relu(self.conv1(x))  # → (batch, out_channels, seq_len)
        x = self.pool(x)              # → (batch, out_channels, 1)
        x = x.squeeze(-1)             # → (batch, out_channels)
        return x

# SIMGPLE FEED FORWARD NEURAL NETWORK 
class PatientEncoder(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 64)
        )

    def forward(self, x):
        return self.encoder(x.float())

# REGRESSION PREDICTION WITH SIMPLE FEED FORWARD NEURAL NETWORK

class DrugResponsePredictor(nn.Module):
    def __init__(self, vocab_size, patient_input_dim):
        super().__init__()
        self.smiles_encoder = SMILESEncoder(vocab_size)
        self.patient_encoder = PatientEncoder(patient_input_dim)
        self.fc = nn.Sequential(
            nn.Linear(64 + 64, 64),
            nn.ReLU(),
            nn.Linear(64, 1)  # IC50 prediction
        )

    def forward(self, smiles_seq, patient_feat):
        drug_vec = self.smiles_encoder(smiles_seq)
        patient_vec = self.patient_encoder(patient_feat)
        combined = torch.cat([drug_vec, patient_vec], dim=1)
        return self.fc(combined).squeeze(-1)


In [100]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from IPython.display import display, HTML

In [102]:
X_train

,Age,SMILES,Cell Name_BT-20,Cell Name_BT-549,Cell Name_CAL-120,Cell Name_CAL-51,Cell Name_CAL-85-1,Cell Name_CAMA-1,Cell Name_HCC1143,Cell Name_HCC1395,...,Race_4.0,Race_5.0,Race_6.0,Race_7.0,Race_8.0,T_stage_by_size_0.0,T_stage_by_size_1.0,T_stage_by_size_2.0,T_stage_by_size_3.0,T_stage_by_size_4.0
35858,-1.433849,C1CC2=CC=CC3=C2N(C1)C=C3[C@H]4[C@@H](C(=O)NC4=...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
90906,0.183106,CCNC(=O)C1=C(C(=C2C=C(C(=CC2=O)O)C(C)C)ON1)C3=...,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
12817,-0.958274,O=C(N[C@H]1CC[C@H](N2CCN(CC3CC3)CC2)CC1)C4=CC=...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
157639,1.704947,CC1=CC2=C(N1)C=CC(=C2F)OC3=NC=NC4=CC(=C(C=C43)...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
157296,-1.338734,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=C(C3=C2)N=C5C=...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,0.658682,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=C(C3=C2)N=C5C=...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
103694,0.563567,C[C@@H]1C[C@H](C2=C1C(=NC=N2)N3CCN(CC3)C(=O)[C...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
131932,-2.099655,CC1=C2[C@H](C(=O)[C@@]3([C@H](C[C@@H]4[C@](C3[...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
146867,1.324487,CC1=CC(=C(C=C1C2CCNCC2)OC(C)C)NC3=NC=C(C(=N3)N...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [104]:
y_train

35858     0.574973
90906     0.008933
12817     0.007348
157639    4.080325
157296    0.009023
            ...   
119879    0.023810
103694    0.204468
131932    0.003317
146867    1.055173
121958    2.240035
Name: EC50 (uM), Length: 174024, dtype: float64

In [106]:
# Data
smiles_list = X_train["SMILES"]  # list of SMILES strings
patient_features = X_train.drop(columns="SMILES").to_numpy()  # (n_samples, patient_dim)
ic50_values = torch.tensor(y_train.to_numpy()).float()  # target IC50

token_to_idx = build_vocab(smiles_list)
smiles_tensor = torch.tensor([smiles_to_indices(sm, token_to_idx) for sm in smiles_list])
patient_tensor = torch.tensor(patient_features).float()
ic50_tensor = ic50_values# torch.tensor(ic50_values).float()

# Model
model = DrugResponsePredictor(vocab_size=len(token_to_idx), patient_input_dim=patient_tensor.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# Training loop (simplified)
model.train()
start_time = time.time() 
for epoch in range(6):
    optimizer.zero_grad()
    preds = model(smiles_tensor, patient_tensor)
    loss = loss_fn(preds, ic50_tensor)
    loss.backward()
    optimizer.step()
    end_time = time.time()
    elapsed = end_time - start_time
    print(f"Epoch {epoch} | Loss: {loss.item():.4f}")
    print(f"Time elapsed: {elapsed} seconds")


Epoch 0 | Loss: 3.9091
Time elapsed: 52.76640796661377 seconds
Epoch 1 | Loss: 3.5575
Time elapsed: 105.81506085395813 seconds
Epoch 2 | Loss: 3.3339
Time elapsed: 158.9662139415741 seconds
Epoch 3 | Loss: 3.2203
Time elapsed: 211.92774081230164 seconds
Epoch 4 | Loss: 3.2027
Time elapsed: 264.8634660243988 seconds
Epoch 5 | Loss: 3.2379
Time elapsed: 317.8196978569031 seconds


In [108]:
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def evaluate_model(model, smiles_list, patient_features, true_ic50s, token_to_idx):
    # Tokenize SMILES
    smiles_indices = [smiles_to_indices(sm, token_to_idx) for sm in smiles_list]
    smiles_tensor = torch.tensor(smiles_indices)

    patient_features = patient_features.to_numpy()
    true_ic50s = true_ic50s.to_numpy()
    
    # Prepare patient features and true labels
    patient_tensor = torch.tensor(patient_features).float()
    true_ic50_tensor = torch.tensor(true_ic50s).float()
    
    with torch.no_grad():
        preds = model(smiles_tensor, patient_tensor).cpu().numpy()
        targets = true_ic50_tensor.cpu().numpy()
    
    mse = mean_squared_error(targets, preds)
    mae = mean_absolute_error(targets, preds)
    r2 = r2_score(targets, preds)
    
    return mse, mae, r2,
    

In [110]:
metrics = evaluate_model(model, X_test["SMILES"], X_test.drop(columns="SMILES"), y_test, token_to_idx)
print(f"mse {metrics[0]}")
print(f"mae {metrics[1]}")
print(f"r2 {metrics[2]}")

mse 3.2127673625946045
mae 1.397216558456421
r2 -0.022051334381103516


In [112]:
def recommend_top_k_drugs_dual_encoder(model, patient_feat, all_smiles, token_to_idx, k=5):
    model.eval()

    # Tokenize SMILES
    smiles_indices = [smiles_to_indices(sm, token_to_idx) for sm in all_smiles]
    smiles_tensor = torch.tensor(smiles_indices)

    # Repeat patient features for all drugs
    patient_tensor = torch.tensor(patient_feat).float().unsqueeze(0).repeat(len(all_smiles), 1)

    # Get predictions
    with torch.no_grad():
        preds = model(smiles_tensor, patient_tensor)

    # Sort & select top-k drugs
    topk = torch.topk(preds, k=k, largest=False)
    topk_indices = topk.indices
    topk_ic50s = topk.values

    # Return SMILES + predicted IC50s
    recommendations = []
    for idx, ic50 in zip(topk_indices, topk_ic50s):
        recommendations.append({
            "smiles": all_smiles[idx],
            "predicted_ic50": ic50.item()
        })

    return recommendations


In [115]:
X

,Age,SMILES,Cell Name_BT-20,Cell Name_BT-549,Cell Name_CAL-120,Cell Name_CAL-51,Cell Name_CAL-85-1,Cell Name_CAMA-1,Cell Name_HCC1143,Cell Name_HCC1395,...,Race_4.0,Race_5.0,Race_6.0,Race_7.0,Race_8.0,T_stage_by_size_0.0,T_stage_by_size_1.0,T_stage_by_size_2.0,T_stage_by_size_3.0,T_stage_by_size_4.0
0,-1.338734,C1C[C@H](CNC1)NC(=O)C2=C(C=C(S2)C3=CC(=CC=C3)F...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,-1.338734,CCN1CCN(CC1)CC2=CN=C(C=C2)NC3=NC=C(C(=N3)C4=CC...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,-1.338734,CC1=C(N=C(N=C1N)[C@H](CC(=O)N)NC[C@@H](C(=O)N)...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,-1.338734,C1COCCN1C2=NC(=NC(=C2)C3=CN=C(C=C3C(F)(F)F)N)N...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,-1.338734,CC1=CC(=C(C=C1C2CCNCC2)OC(C)C)NC3=NC=C(C(=N3)N...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217525,1.324487,CC[C@@]1(C2=C(COC1=O)C(=O)N3CC4=C(C3=C2)N=C5C=...,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
217526,1.324487,C1=CC(=CC(=C1)N2C(=O)C=CC3=CN=C4C=CC(=CC4=C32)...,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
217527,1.324487,CC1=C2C(=C(N(C1=O)C)NC3=C(C=C(C=C3)I)F)C(=O)N(...,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
217528,1.324487,O=C(N[C@H]1CC[C@H](N2CCN(CC3CC3)CC2)CC1)C4=CC=...,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [121]:
# Recommend for a given patient
patient_i = X.drop(columns="SMILES").iloc[80].to_numpy()
top5 = recommend_top_k_drugs_dual_encoder(model, patient_i, smiles_list.tolist(), token_to_idx)

In [122]:
top5

[{'smiles': 'N.N.[Cl-].[Cl-].[Pt+2]', 'predicted_ic50': 0.9994981288909912},
 {'smiles': 'N.N.[Cl-].[Cl-].[Pt+2]', 'predicted_ic50': 0.9994981288909912},
 {'smiles': 'N.N.[Cl-].[Cl-].[Pt+2]', 'predicted_ic50': 0.9994981288909912},
 {'smiles': 'N.N.[Cl-].[Cl-].[Pt+2]', 'predicted_ic50': 0.9994981288909912},
 {'smiles': 'N.N.[Cl-].[Cl-].[Pt+2]', 'predicted_ic50': 0.9994981288909912}]

# ^ EVERY PATIENT'S PREDICTION IS THIS DRUG WHICH DOES NOT SEEM CORRECT